<b> Step 1: 1st Year Data Collection </b>

In [1]:
# Setting up webscraper to grab the table where data is housed
from bs4 import BeautifulSoup
import requests

year = 2023
url = 'https://www.capfriendly.com/browse/active/%s/salary?display=signing-team,birthday,weight,height,draft,caphit-percent,length,skater-individual-advanced-stats,skater-on-ice-advanced-stats,signing-age&hide=caphit,goalie-stats' % year
rq = requests.get(url)
soup = BeautifulSoup(rq.text, features='html.parser')

col_names = soup.find_all('th')
pd_columns = [ title.text for title in col_names ]
print(pd_columns)

['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'DRAFTED', 'GP', 'G', 'A', 'P', 'P/GP', '+/-', 'Sh', 'Sh%', 'TOI', 'ixG', 'iSh', 'iCF', 'iFF', 'ixG60', 'iSh60', 'iCF60', 'iFF60', 'SF', 'SA', 'SF%', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'xGF', 'xGA', 'xGF%', 'SIGNING TEAM', 'SIGNING AGE', 'CLAUSE', 'LENGTH', 'EXPIRY', 'CAP HIT %', 'SALARY']


In [2]:
# Setting up pandas dataframe to collect & organize the scraped data
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn' # clearing false-positive warning that will show executing later loop

df = pd.DataFrame(columns = pd_columns)
df

,PLAYER,TEAM,AGE,DATE OF BIRTH,WEIGHT,HEIGHT,POS,HANDED,DRAFTED,GP,...,xGF,xGA,xGF%,SIGNING TEAM,SIGNING AGE,CLAUSE,LENGTH,EXPIRY,CAP HIT %,SALARY


In [3]:
# Taking data from HTML table & inserting it into df
column_data = soup.find_all('tr')
for row in column_data[1:]:
    row_data = row.find_all('td')
    idv_row_data = [data.text for data in row_data]

    length = len(df)
    df.loc[length] = idv_row_data    
print(df.head())
print(len(df))

                 PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT    POS HANDED  \
0       1. Tyler Seguin  DAL  30  Jan. 31, 1992    200   6'1"  C, RW  Right   
1     2. Artemi Panarin  NYR  30  Oct. 30, 1991    170  5'11"     LW  Right   
2  3. Aleksander Barkov  FLA  26   Sep. 2, 1995    210   6'3"      C   Left   
3      4. Darnell Nurse  EDM  27   Feb. 4, 1995    221   6'4"     LD   Left   
4   5. Sergei Bobrovsky  FLA  33  Sep. 20, 1988    187   6'2"      G   Left   

                    DRAFTED  GP  ...     xGF     xGA   xGF% SIGNING TEAM  \
0  2 - Round 1 - 2010 (BOS)  76  ...   79.05   56.68  58.24          DAL   
1                         -  82  ...  113.85   66.27  63.21          NYR   
2  2 - Round 1 - 2013 (FLA)  68  ...  112.68   74.51  60.20          FLA   
3  7 - Round 1 - 2013 (EDM)  82  ...  101.95  106.56  48.89          EDM   
4                         -  50  ...       -       -      -          FLA   

  SIGNING AGE CLAUSE LENGTH EXPIRY CAP HIT %       SALARY  
0       

In [4]:
# Setting up loop to collect player data from all each page of the table
import re
last_pg = str(soup.find_all('a', string='Last')).split()[3]
pattern = r'"(.*?)"'
loop_len = int(re.findall(pattern, last_pg)[0])-1
print(loop_len)

loops = 0
pg = 2
while (loops < loop_len):
    # URL changes after 1st page to track the depth into the table, allowing us to measurably increment and pull the entire table
    url = 'https://www.capfriendly.com/browse/active/%s/salary?stats-season=2023&display=signing-team,birthday,weight,height,draft,caphit-percent,length,skater-individual-advanced-stats,skater-on-ice-advanced-stats,signing-age&hide=caphit,goalie-stats&pg=%s' % (year,pg)
    rq = requests.get(url)
    soup = BeautifulSoup(rq.text, features='html.parser')

    column_data = soup.find_all('tr')

    for row in column_data[1:]:
        row_data = row.find_all('td')
        idv_row_data = [data.text for data in row_data]

        length = len(df)
        df.loc[length] = idv_row_data    

    pg = pg + 1
    loops = loops + 1
print(df.head())
print(len(df))

31
                 PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT    POS HANDED  \
0       1. Tyler Seguin  DAL  30  Jan. 31, 1992    200   6'1"  C, RW  Right   
1     2. Artemi Panarin  NYR  30  Oct. 30, 1991    170  5'11"     LW  Right   
2  3. Aleksander Barkov  FLA  26   Sep. 2, 1995    210   6'3"      C   Left   
3      4. Darnell Nurse  EDM  27   Feb. 4, 1995    221   6'4"     LD   Left   
4   5. Sergei Bobrovsky  FLA  33  Sep. 20, 1988    187   6'2"      G   Left   

                    DRAFTED  GP  ...     xGF     xGA   xGF% SIGNING TEAM  \
0  2 - Round 1 - 2010 (BOS)  76  ...   79.05   56.68  58.24          DAL   
1                         -  82  ...  113.85   66.27  63.21          NYR   
2  2 - Round 1 - 2013 (FLA)  68  ...  112.68   74.51  60.20          FLA   
3  7 - Round 1 - 2013 (EDM)  82  ...  101.95  106.56  48.89          EDM   
4                         -  50  ...       -       -      -          FLA   

  SIGNING AGE CLAUSE LENGTH EXPIRY CAP HIT %       SALARY  
0    

<b> Step 2: Data Manipulation </b>

In [5]:
# Removing Goalie Data for data cleaning
df_no_goalies = df.loc[df['POS'] != 'G']

# Correcting numeric attributes' dtypes
to_numeric = ['WEIGHT','GP','G','A','P','+/-','Sh','SF','SA','CF','CA','FF','FA','iSh','iCF','iFF','SIGNING AGE','LENGTH','P/GP','Sh%','ixG','ixG60','iSh60','iCF60','iFF60','SF%','CF%','FF%','xGF','xGA','xGF%']
df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)

# Cases needing additional transformation
ht_list = df_no_goalies['HEIGHT'].str.split("'")
feet = ht_list.str[0].astype(int)
inches = ht_list.str[1].str.replace('"', '').astype(int)
total_in = (feet*12) + inches
df_no_goalies['HEIGHT'] = total_in
df_no_goalies.dtypes

ts = df_no_goalies['TOI'].replace('','0:0').str.split(':')
mn = ts.str[0].astype(int)
sc = ts.str[1].astype(int)
total_avg_sec = (mn*60) + sc
df_no_goalies['TOI'] = total_avg_sec

age = df_no_goalies['AGE'].str.extract(pat='(\d+)', expand=False).astype(int)
df_no_goalies['AGE'] = age

salary = df_no_goalies['SALARY'].str.replace('[^0-9]','').astype(int)
df_no_goalies['SALARY'] = salary

caph = df_no_goalies['CAP HIT %'].str.rstrip('%').astype(float)
df_no_goalies['CAP HIT %'] = caph

# Cleaning up the PlAYER attribute to show only player name
df_no_goalies['PLAYER'] = df_no_goalies['PLAYER'].str.lstrip('1234567890. ')

C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1429756543.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1429756543.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies['HEIGHT'] = total_in
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1429756543.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

<b> Step 3: Exporting dataframe to .CSV </b>

In [6]:
file_name = 'NHL_skaters_%s.csv' % year
df_no_goalies.to_csv(file_name)

print('%s Completed & exported' % year)

2023 Completed & exported


<b> Step 4: *FINAL SCRIPT* - Introducing loop to collect all available years of advanced-stat tracking </b>

In [7]:
# Setting up webscraper to grab the table where data is housed
from bs4 import BeautifulSoup
import requests

year = 2023
while year > 2008:
    url = 'https://www.capfriendly.com/browse/active/%s/salary?display=signing-team,birthday,weight,height,draft,caphit-percent,length,skater-individual-advanced-stats,skater-on-ice-advanced-stats,signing-age&hide=caphit,goalie-stats' % year
    rq = requests.get(url)
    soup = BeautifulSoup(rq.text, features='html.parser')

    col_names = soup.find_all('th')
    pd_columns = [ title.text for title in col_names ]
    print(pd_columns)

    # Setting up pandas dataframe to collect & organize the scraped data
    import pandas as pd

    df = pd.DataFrame(columns = pd_columns)
    df

    # Taking data from HTML table & inserting it into df
    column_data = soup.find_all('tr')
    for row in column_data[1:]:
        row_data = row.find_all('td')
        idv_row_data = [data.text for data in row_data]

        length = len(df)
        df.loc[length] = idv_row_data    
    print(df.head())
    print(len(df))

    # Setting up loop to collect player data from all each page of the table
    import re
    last_pg = str(soup.find_all('a', string='Last')).split()[3]
    pattern = r'"(.*?)"'
    loop_len = int(re.findall(pattern, last_pg)[0])-1
    print(loop_len)

    loops = 0
    pg = 2
    while (loops < loop_len):
        # URL changes after 1st page to track the depth into the table, allowing us to measurably increment and pull the entire table
        url = 'https://www.capfriendly.com/browse/active/%s/salary?stats-season=2023&display=signing-team,birthday,weight,height,draft,caphit-percent,length,skater-individual-advanced-stats,skater-on-ice-advanced-stats,signing-age&hide=caphit,goalie-stats&pg=%s' % (year,pg)
        rq = requests.get(url)
        soup = BeautifulSoup(rq.text, features='html.parser')

        column_data = soup.find_all('tr')

        for row in column_data[1:]:
            row_data = row.find_all('td')
            idv_row_data = [data.text for data in row_data]

            length = len(df)
            df.loc[length] = idv_row_data    

        pg = pg + 1
        loops = loops + 1
    print(df.head())
    print(len(df))

    # Removing Goalie Data for data cleaning
    df_no_goalies = df.loc[df['POS'] != 'G']

    # Correcting numeric attributes' dtypes
    to_numeric = ['WEIGHT','GP','G','A','P','+/-','Sh','SF','SA','CF','CA','FF','FA','iSh','iCF','iFF','SIGNING AGE','LENGTH','P/GP','Sh%','ixG','ixG60','iSh60','iCF60','iFF60','SF%','CF%','FF%','xGF','xGA','xGF%']
    df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)

    # Cases needing additional transformation
    ht_list = df_no_goalies['HEIGHT'].str.split("'")
    feet = ht_list.str[0].astype(int)
    inches = ht_list.str[1].str.replace('"', '').astype(int)
    total_in = (feet*12) + inches
    df_no_goalies['HEIGHT'] = total_in
    df_no_goalies.dtypes

    ts = df_no_goalies['TOI'].replace('','0:0').str.split(':')
    mn = ts.str[0].astype(int)
    sc = ts.str[1].astype(int)
    total_avg_sec = (mn*60) + sc
    df_no_goalies['TOI'] = total_avg_sec

    age = df_no_goalies['AGE'].str.extract(pat='(\d+)', expand=False).astype(int)
    df_no_goalies['AGE'] = age

    salary = df_no_goalies['SALARY'].str.replace('[^0-9]','').astype(int)
    df_no_goalies['SALARY'] = salary

    caph = df_no_goalies['CAP HIT %'].str.rstrip('%').astype(float)
    df_no_goalies['CAP HIT %'] = caph

    # Cleaning up the PlAYER attribute to show only player name
    df_no_goalies['PLAYER'] = df_no_goalies['PLAYER'].str.lstrip('1234567890. ')

    file_name = 'NHL_skaters_%s.csv' % year
    df_no_goalies.to_csv(file_name)

    print('%s Completed & exported' % year)
    year = year - 1

['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'DRAFTED', 'GP', 'G', 'A', 'P', 'P/GP', '+/-', 'Sh', 'Sh%', 'TOI', 'ixG', 'iSh', 'iCF', 'iFF', 'ixG60', 'iSh60', 'iCF60', 'iFF60', 'SF', 'SA', 'SF%', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'xGF', 'xGA', 'xGF%', 'SIGNING TEAM', 'SIGNING AGE', 'CLAUSE', 'LENGTH', 'EXPIRY', 'CAP HIT %', 'SALARY']
                 PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT    POS HANDED  \
0       1. Tyler Seguin  DAL  30  Jan. 31, 1992    200   6'1"  C, RW  Right   
1     2. Artemi Panarin  NYR  30  Oct. 30, 1991    170  5'11"     LW  Right   
2  3. Aleksander Barkov  FLA  26   Sep. 2, 1995    210   6'3"      C   Left   
3      4. Darnell Nurse  EDM  27   Feb. 4, 1995    221   6'4"     LD   Left   
4   5. Sergei Bobrovsky  FLA  33  Sep. 20, 1988    187   6'2"      G   Left   

                    DRAFTED  GP  ...     xGF     xGA   xGF% SIGNING TEAM  \
0  2 - Round 1 - 2010 (BOS)  76  ...   79.05   56.68  58.24          DAL   

C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies['HEIGHT'] = total_in
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'DRAFTED', 'GP', 'G', 'A', 'P', 'P/GP', '+/-', 'Sh', 'Sh%', 'TOI', 'ixG', 'iSh', 'iCF', 'iFF', 'ixG60', 'iSh60', 'iCF60', 'iFF60', 'SF', 'SA', 'SF%', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'xGF', 'xGA', 'xGF%', 'SIGNING TEAM', 'SIGNING AGE', 'CLAUSE', 'LENGTH', 'EXPIRY', 'CAP HIT %', 'SALARY']
              PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT    POS HANDED  \
0   1. Erik Karlsson  PIT  31   May 31, 1990    190   6'0"     RD  Right   
1  2. Artemi Panarin  NYR  29  Oct. 30, 1991    170  5'11"     LW  Right   
2    3. Tyler Seguin  DAL  29  Jan. 31, 1992    200   6'1"  C, RW  Right   
3  4. Connor McDavid  EDM  24  Jan. 13, 1997    193   6'1"      C   Left   
4     5. Carey Price  MTL  33  Aug. 16, 1987    217   6'3"      G   Left   

                     DRAFTED  GP  ...     xGF     xGA   xGF% SIGNING TEAM  \
0  15 - Round 1 - 2008 (OTT)  82  ...  137.16  111.12  55.24          SJS   
1              

C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies['HEIGHT'] = total_in
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'DRAFTED', 'GP', 'G', 'A', 'P', 'P/GP', '+/-', 'Sh', 'Sh%', 'TOI', 'ixG', 'iSh', 'iCF', 'iFF', 'ixG60', 'iSh60', 'iCF60', 'iFF60', 'SF', 'SA', 'SF%', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'xGF', 'xGA', 'xGF%', 'SIGNING TEAM', 'SIGNING AGE', 'CLAUSE', 'LENGTH', 'EXPIRY', 'CAP HIT %', 'SALARY']
                 PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT POS HANDED  \
0    1. Auston Matthews  TOR  22  Sep. 17, 1997    220   6'3"   C   Left   
1    2. Mitchell Marner  TOR  23    May 5, 1997    175   6'0"  RW  Right   
2     3. Connor McDavid  EDM  23  Jan. 13, 1997    193   6'1"   C   Left   
3     4. Artemi Panarin  NYR  28  Oct. 30, 1991    170  5'11"  LW  Right   
4  5. Nicklas Bäckström  WSH  32  Nov. 23, 1987    206   6'1"   C   Left   

                    DRAFTED  GP  ...     xGF    xGA   xGF% SIGNING TEAM  \
0  1 - Round 1 - 2016 (TOR)  74  ...  124.32  69.62  64.09          TOR   
1  4 - Round 1 - 20

...

C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies['HEIGHT'] = total_in
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'DRAFTED', 'GP', 'G', 'A', 'P', 'P/GP', '+/-', 'Sh', 'Sh%', 'TOI', 'ixG', 'iSh', 'iCF', 'iFF', 'ixG60', 'iSh60', 'iCF60', 'iFF60', 'SF', 'SA', 'SF%', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'xGF', 'xGA', 'xGF%', 'SIGNING TEAM', 'SIGNING AGE', 'CLAUSE', 'LENGTH', 'EXPIRY', 'CAP HIT %', 'SALARY']
                  PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT    POS HANDED  \
0  1. Vincent Lecavalier    -  29  Apr. 21, 1980    215   6'4"  C, LW   Left   
1       2. Alex Ovechkin  WSH  23  Sep. 17, 1985    236   6'3"     LW  Right   
2       3. Evgeni Malkin  PIT  22  Jul. 31, 1986    195   6'3"      C   Left   
3       4. Sidney Crosby  PIT  21   Aug. 7, 1987    200  5'11"      C   Left   
4         5. Chris Drury    -  32  Aug. 20, 1976    190  5'10"      C  Right   

                     DRAFTED  GP  ...     xGF    xGA   xGF% SIGNING TEAM  \
0   1 - Round 1 - 1998 (TBL)   0  ...    0.00   0.00   0.00          

C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies['HEIGHT'] = total_in
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

['PLAYER', 'TEAM', 'AGE', 'DATE OF BIRTH', 'WEIGHT', 'HEIGHT', 'POS', 'HANDED', 'DRAFTED', 'GP', 'G', 'A', 'P', 'P/GP', '+/-', 'Sh', 'Sh%', 'TOI', 'ixG', 'iSh', 'iCF', 'iFF', 'ixG60', 'iSh60', 'iCF60', 'iFF60', 'SF', 'SA', 'SF%', 'CF', 'CA', 'CF%', 'FF', 'FA', 'FF%', 'xGF', 'xGA', 'xGF%', 'SIGNING TEAM', 'SIGNING AGE', 'CLAUSE', 'LENGTH', 'EXPIRY', 'CAP HIT %', 'SALARY']
                PLAYER TEAM AGE  DATE OF BIRTH WEIGHT HEIGHT POS HANDED  \
0      1. Dany Heatley    -  27  Jan. 21, 1981    212   6'3"  LW   Left   
1     2. Alex Ovechkin  WSH  22  Sep. 17, 1985    236   6'3"  LW  Right   
2     3. Sidney Crosby  PIT  20   Aug. 7, 1987    200  5'11"   C   Left   
3  4. Miikka Kiprusoff    -  31  Oct. 26, 1976    183   6'1"   G   Left   
4      5. Jaromír Jágr    -  36  Feb. 15, 1972    230   6'3"  RW   Left   

                      DRAFTED  GP  ...     xGF    xGA   xGF% SIGNING TEAM  \
0    2 - Round 1 - 2000 (ATL)   0  ...    0.00   0.00   0.00          OTT   
1    1 - Round 1 - 20

C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies[to_numeric] = df_no_goalies[to_numeric].apply(pd.to_numeric)
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_goalies['HEIGHT'] = total_in
C:\Users\logan\AppData\Local\Temp\ipykernel_9844\1580694076.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro